In [1]:
class Node(object):

    def __init__(self, value):
        self.value = value
        # A dictionary is used for neighbors. keys are neighbor nodes keys, 
        # values are weights of arcs between nodes
        self.neighbors = {}
        self.inDegree = 0
        self.outDegree = 0

    def updateEdgeWeight(self, key):
        pass

    def get_edge_weight(self, key):
        if self.isConnectedTo(key):
            return self.neighbors[key]
        return None

    def connectTo(self, key, weight):
        self.neighbors[key] = weight

    def isConnectedTo(self, key):
        return key in self.neighbors


class Graph(object):

    def __init__(self, directed_graph=False):

        self.directed_graph = directed_graph
        self.nodes = {}
        self.nodesCount = 0
        self.edgeCount = 0


    def add_node(self, key, value):
        if not self.has_node(key):
            self.nodes[key] = Node(value)
            self.nodesCount += 1

    def add_edge(self, key1, key2, weight=1):
        if not self.has_node(key1) or not self.has_node(key2):
            return None
        if not self.has_edge(key1, key2):
            self.nodes[key1].connectTo(key2, weight)
            self.edgeCount += 1

        if not self.directed_graph:
            if not self.has_edge(key2, key1):
                self.nodes[key2].connectTo(key1, weight)


    def get_node_value(self, key):

        if self.has_node(key):
            return self.nodes[key].value
        return None

    def get_edge_weight(self, key1, key2):
        if self.has_node(key1):
            return self.nodes[key1].get_edge_weight(key2)
        return None

    def get_all_nodes(self):
        res = []
        for k in self.nodes:
            res.append((k, self.get_node_value(k)))
        return res


    def get_all_edges(self):
        res = []
        for k in self.nodes:
            for k1 in self.nodes[k].neighbors:
                if self.directed_graph:
                    res.append((k, k1))
                else:
                    if (k1, k) not in res:
                        res.append((k, k1))
        return res

    def get_neighbours(self, key):
        if self.has_node(key):
            return [(x, self.get_node_value(x)) for x in self.nodes[key].neighbors]
        return None

    def has_node(self, key):
        """
        Returns Tru
        e if node with 'key' is in the graph
        """
        return key in self.nodes

    def has_edge(self, key1, key2):


        if self.has_node(key1):
            return self.nodes[key1].isConnectedTo(key2)
        return False

    # ALGORITHMS

    

    def get_graph_matrix(self):
        mapKeysToIndexes = {}
        mapIndexesToKeys = [None] * self.nodesCount
        c = 0
        for k in self.nodes:
            mapKeysToIndexes[k] = c
            mapIndexesToKeys[c] = k
            c += 1

        matrix = [[0 for _ in range(self.nodesCount)]
                  for _ in range(self.nodesCount)]
        for k in self.nodes:
            for k1 in self.nodes[k].neighbors:
                matrix[mapKeysToIndexes[k]][mapKeysToIndexes[k1]] = \
                    self.nodes[k].neighbors[k1]

        return matrix, mapIndexesToKeys

   
    def __str__(self):
        s = ""
        for n1 in self.nodes:
            s += str(n1) + ": "
            for n2 in self.nodes[n1].neighbors:
                s += str(n2) + "->"
            s += "\n"
        return s



In [2]:
class MarkovClustering(object):

    def __init__(self, matrix, e, r):
        self.matrix = numpy.array(matrix, dtype=numpy.float64)
        self.e = e
        self.r = r

    def compute_clusters(self, T=100):
        self.add_self_loops()
        self.normalize_columns()

        t = 0
        while t < T:
            last_matrix = numpy.copy(self.matrix)
            self.power_step()
            self.inflation_step()
            if self.steady_state(last_matrix) == True:
                break
            t += 1

        return self.interpret_clusters()

    def add_self_loops(self):
        for i in range(self.matrix.shape[0]):
            self.matrix[i][i] = 1

    def normalize_columns(self):
        s = self.matrix.sum(axis=0)
        for (x, y), _ in numpy.ndenumerate(self.matrix):
            if s[y] != 0:
                self.matrix[x][y] /= float(s[y])

    def power_step(self):
        temp = self.matrix
        for _ in range(self.e - 1):
            temp = temp.dot(self.matrix)
        self.matrix = temp

    def inflation_step(self):
        self.matrix **= self.r
        self.normalize_columns();

    def steady_state(self, last_matrix):
        for (x, y), _ in numpy.ndenumerate(self.matrix):
            if self.matrix[x][y] - last_matrix[x][y] != 0:
                return False
        return True

    def interpret_clusters(self):
        res = []
        for i in range(self.matrix.shape[0]):
            cluster = []
            flag = 0
            for z in range(self.matrix.shape[0]):
                if self.matrix[i][z] > 0:
                    cluster.append(z)
                    flag = 1
            if flag == 1:
                res.append(cluster)
        return res


In [1]:
import numpy

f = open("../data/data.txt", "r")

grafo = Graph()

for line in f:
    edges = line.split()
    grafo.add_node(edges[0], edges[0])
    grafo.add_node(edges[1], edges[1])
    grafo.add_edge(edges[0], edges[1])

print("Graph adjacency list representation: \n", grafo)

matrix, mapBackToKeys = grafo.get_graph_matrix()
numpymat = numpy.array(matrix)
print("Graph matrix representation: \n", numpymat)
print("\nMapping matrix indexes to keys: ", mapBackToKeys)


alg = MarkovClustering(matrix, e=2, r=2)
clusters = alg.compute_clusters(T=40)

print("\nClusters after MCL algorithm: ")
for cluster in clusters:
    print([mapBackToKeys[x] for x in cluster])


NameError: name 'Graph' is not defined